In [28]:
import SimpleITK as sitk
import numpy as np
import os
import matplotlib.pyplot as plt

In [6]:
baseDir = os.getcwd()
patientNumber = "01"

preopDrrPath = os.path.join(baseDir, f"input_files/pacient_{patientNumber}/pacient{patientNumber}Preop.mha")
intraopDrrPath =  os.path.join(baseDir, f"input_files/pacient_{patientNumber}/pacient{patientNumber}Intraop.mha")

preopImage = sitk.ReadImage(preopDrrPath, sitk.sitkFloat32)
intraopImage = sitk.ReadImage(intraopDrrPath, sitk.sitkFloat32)

intraopImageInverted = sitk.InvertIntensity(intraopImage, maximum=1)
print(f"moving: {preopImage.GetSize()}, fixed: {intraopImageInverted.GetSize()}")

fixedImage = intraopImageInverted
movingImage = preopImage[:, :, 0]
movingImage.SetOrigin(fixedImage.GetOrigin())
movingImage.SetSpacing(fixedImage.GetSpacing())
print(f"Fixed image, spacing: {fixedImage.GetSpacing()}, size: {fixedImage.GetSize()}, direction: {fixedImage.GetDirection()}, origin: {fixedImage.GetOrigin()}")
print(f"Moving image, spacing: {movingImage.GetSpacing()}, size: {movingImage.GetSize()}, direction: {movingImage.GetDirection()}, origin: {movingImage.GetOrigin()}")
plt.imshow(sitk.GetArrayViewFromImage(fixedImage), cmap="gray")
plt.imshow(sitk.GetArrayViewFromImage(movingImage), cmap="gray", alpha=0.5)
plt.axis("off")
plt.show()

('c:\\Users\\vojte\\Desktop\\skola\\DP\\input_files/pacient_01/pacient01Preop.mha',
 'c:\\Users\\vojte\\Desktop\\skola\\DP\\input_files/pacient_01/pacient01Intraop.mha')

In [20]:
def palin(inputText):
    return True if inputText == inputText[::-1] else False

In [ ]:
initialTransform = sitk.CenteredTransformInitializer(fixedImage, movingImage,
                                                     sitk.AffineTransform(2))
registration = sitk.ImageRegistrationMethod()
registration.SetMetricAsMattesMutualInformation()
registration.SetOptimizerScalesFromPhysicalShift()
registration.SetOptimizerAsGradientDescent(learningRate=1.0,
                                           numberOfIterations=200,
                                           convergenceMinimumValue=1e-5,
                                           convergenceWindowSize=5)
registration.SetInitialTransform(initialTransform)
registration.SetInterpolator(sitk.sitkLinear)

#registration.SetShrinkFactorsPerLevel(shrinkFactors=[4, 2, 1])
#registration.SetSmoothingSigmasPerLevel(smoothingSigmas=[2, 1, 0])
#registration.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()

registration.AddCommand(sitk.sitkStartEvent, start_plot)
registration.AddCommand(sitk.sitkEndEvent, end_plot)
#registration.AddCommand(sitk.sitkMultiResolutionIterationEvent, update_multires_iterations)
registration.AddCommand(sitk.sitkIterationEvent, lambda: plot_values(registration))
outTransform = registration.Execute(fixedImage, movingImage)
print(f"Optimizer stop condition: {registration.GetOptimizerStopConditionDescription()}")
print(f" Iteration: {registration.GetOptimizerIteration()}")
print(f" Metric value: {registration.GetMetricValue()}")

plt.plot(range(len(metric_values)), metric_values)
plt.xlabel("Iterace")
plt.ylabel("Mutual information")
plt.imshow()